# Finale Aufgabe für Praktikum Deep Learning <br>Textgenerierung mit RNN: Modelltraining

* **Name:** Fabian Schotte
* **Email:** fabian.schotte@rwu.de
* **Matrikelnummer:** 35604
* **Studiengang:** Angewandte Informatik

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import os
import matplotlib.pylab as plt
from work import models
import time 

os.makedirs("models", exist_ok=True)

## Vorbereitung


### Laden der Trainingsdaten
Hier werden die Trainings- und Testdaten der Kaggle Sentiment Analyis aus deren CSV-Dateien ausgelesen und die Inhalte der Spalte `text` zu dem String `kaggle_text` zusammengefasst.

In [3]:
df_train = pd.read_csv('work/kaggle_sentiment/tweet_sentiment_train.csv', encoding='utf-8', encoding_errors='replace')
df_test = pd.read_csv('work/kaggle_sentiment/tweet_sentiment_test.csv', encoding='utf-8', encoding_errors='replace')

kaggle_text_train = df_train['text'].str.cat(sep='\n')
kaggle_text_test = df_test['text'].str.cat(sep='\n')
# kaggle_text = kaggle_text_train + '\n' + kaggle_text_test
kaggle_text = kaggle_text_train
# kaggle_text = kaggle_text_test

print(kaggle_text[:500])

 I`d have responded, if I were going
 Sooo SAD I will miss you here in San Diego!!!
my boss is bullying me...
 what interview! leave me alone
 Sons of ****, why couldn`t they put them on the releases we already bought
http://www.dothebouncy.com/smf - some shameless plugging for the best Rangers forum on earth
2am feedings for the baby are fun when he is all smiles and coos
Soooo high
 Both of you
 Journey!? Wow... u just became cooler.  hehe... (is that possible!?)
 as much as i love to be hopef


Im nächsten Codeblock wird ein Set der einzigartigen Charaktere im String `kaggle_text` mit dem Namen `vocab` erstellt. Ebenso werden die darin vorhandenen Charaktere ausgegeben und die Länge des Sets ausgegeben.

In [4]:
vocab = sorted(set(kaggle_text))
print(vocab)
print(f"vocab size = {len(vocab)}")

['\t', '\n', ' ', '!', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '^', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}', '~', '\xa0', '´', '½', '¿', 'Â', 'ï']
vocab size = 102


## Preprocessing
Im folgenden Codeblock wird ein Beispieltext zu einer Liste von Charakteren aufgeteilt und ausgegeben. Diese List wird als `chars` gespeichert.

In [7]:
example_texts = ['hello world', 'hello world']
chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'h', b'e', b'l', b'l', b'o', b' ', b'w', b'o', b'r', b'l', b'd'],
 [b'h', b'e', b'l', b'l', b'o', b' ', b'w', b'o', b'r', b'l', b'd']]>

Hier werden 

In [8]:
ids_from_chars = keras.layers.StringLookup(vocabulary=list(vocab), mask_token=None)
ids = ids_from_chars(chars)
print(ids)

chars_from_ids = keras.layers.StringLookup(vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)
chars = chars_from_ids(ids)
print(chars)

tf.strings.reduce_join(chars, axis=-1).numpy()

<tf.RaggedTensor [[74, 71, 78, 78, 81, 3, 89, 81, 84, 78, 70],
 [74, 71, 78, 78, 81, 3, 89, 81, 84, 78, 70]]>
<tf.RaggedTensor [[b'h', b'e', b'l', b'l', b'o', b' ', b'w', b'o', b'r', b'l', b'd'],
 [b'h', b'e', b'l', b'l', b'o', b' ', b'w', b'o', b'r', b'l', b'd']]>


array([b'hello world', b'hello world'], dtype=object)

In [ ]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [ ]:
all_ids = ids_from_chars(tf.strings.unicode_split(kaggle_text, 'UTF-8'))
all_ids

In [ ]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

In [ ]:
seq_length = 100

In [ ]:
sequences = ids_dataset.batch(seq_length + 1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq).numpy())
for seq in sequences.take(1):
  print(text_from_ids(seq).numpy())

In [ ]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [ ]:
# split_input_target(list("Tensorflow"))

In [ ]:
dataset = sequences.map(split_input_target)
len(dataset)

In [ ]:
for input_example, target_example in dataset.take(1):
    print("Input:", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())
    print("Input shape:", input_example.shape)
    print("Target shape:", target_example.shape)

In [ ]:
BATCH_SIZE = 150
BUFFER_SIZE = 1000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
)
dataset

In [ ]:
vocab_size = len(ids_from_chars.get_vocabulary())
embedding_dim = 256
rnn_units = 2048

### 1. GRU-Modell

In [ ]:
vocab_size_1 = len(ids_from_chars.get_vocabulary())
embedding_dim_1 = 256
rnn_units_1 = 2048

In [ ]:
inputs_1 = keras.layers.Input(shape=(None,), dtype='int32', name='input_tokens')
embedding_1 = keras.layers.Embedding(input_dim=vocab_size_1, output_dim=embedding_dim_1)(inputs_1)
gru_1, gru_state_1 = keras.layers.GRU(units=rnn_units_1, return_sequences=True, return_state=True)(embedding_1)
gru_1, gru_state_1 = keras.layers.GRU(units=rnn_units_1, return_sequences=True, return_state=True)(gru_1)
outputs_1 = keras.layers.Dense(units=vocab_size, activation='softmax')(gru_1)

gru_model_1 =  keras.Model(inputs=inputs_1, outputs=outputs_1)

#### Testen des Modells

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions_model_1 = gru_model_1(input_example_batch)
    print(example_batch_predictions_model_1.shape, "# (batch_size, sequence_length, vocab_size)")

In [ ]:
gru_model_1.summary()

In [ ]:
sampled_indices_model_1 = tf.random.categorical(example_batch_predictions_model_1[0], num_samples=1)
sampled_indices_model_1 = tf.squeeze(sampled_indices_model_1, axis=-1).numpy()
sampled_indices_model_1

In [ ]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices_model_1).numpy())

### Training

#### Loss

In [ ]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
example_batch_mean_loss_model_1 = loss(target_example_batch, example_batch_predictions_model_1)
print("Prediction shape: ", example_batch_predictions_model_1.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss_model_1)

In [ ]:
tf.exp(example_batch_mean_loss_model_1).numpy()

#### Optimizer

In [ ]:
gru_model_1.compile(optimizer='adam', loss=loss, metrics=['accuracy'], run_eagerly=True)

#### Early Stopping

In [ ]:
early_stopping_gru_model_1 = keras.callbacks.EarlyStopping(monitor="loss", min_delta=0.002, patience=2)

#### Konfiguration von Checkpoints

In [ ]:
checkpoint_dir = './work/training_checkpoints/gru_model_1'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}.weights.h5")

checkpoint_callback_gru_model_1=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

#### Ausführen des Trainings

In [ ]:
EPOCHS = 30

In [ ]:
start = time.perf_counter()
gru_model_1_history = gru_model_1.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback_gru_model_1, early_stopping_gru_model_1])
end = time.perf_counter()
gru_model_1_training_time = end - start

#### Speichern

In [ ]:
gru_model_1.save('work/models/gru_model_1.keras')

### 2. GRU-Modell

In [ ]:
vocab_size_2 = len(ids_from_chars.get_vocabulary())
embedding_dim_2 = 256
rnn_units_2 = 1024

In [ ]:
inputs_2 = keras.layers.Input(shape=(None,), dtype='int32', name='input_tokens')
embedding_2 = keras.layers.Embedding(input_dim=vocab_size_2, output_dim=embedding_dim_2)(inputs_2)
gru_2, gru_state_2 = keras.layers.GRU(units=rnn_units_2, return_sequences=True, return_state=True)(embedding_2)
dropout_2 = keras.layers.Dropout(0.2)(gru_2)
gru_2, gru_state_2 = keras.layers.GRU(units=rnn_units_2, return_sequences=True, return_state=True)(dropout_2)
outputs_2 = keras.layers.Dense(units=vocab_size, activation='softmax')(gru_2)

gru_model_2 = keras.Model(inputs=inputs_2, outputs=outputs_2)

#### Testen des Modells

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions_gru_model_2 = gru_model_2(input_example_batch)
    print(example_batch_predictions_gru_model_2.shape, "# (batch_size, sequence_length, vocab_size)")

In [ ]:
gru_model_2.summary()

In [ ]:
sampled_indices_gru_model_2 = tf.random.categorical(example_batch_predictions_gru_model_2[0], num_samples=1)
sampled_indices_gru_model_2 = tf.squeeze(sampled_indices_gru_model_2, axis=-1).numpy()
sampled_indices_gru_model_2

In [ ]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices_gru_model_2).numpy())

### Training

#### Loss

In [ ]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
example_batch_mean_loss_model_2 = loss(target_example_batch, example_batch_predictions_gru_model_2)
print("Prediction shape: ", example_batch_predictions_gru_model_2.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss_model_2)

In [ ]:
tf.exp(example_batch_mean_loss_model_1).numpy()

#### Optimizer

In [ ]:
gru_model_2.compile(optimizer='adam', loss=loss, metrics=['accuracy'], run_eagerly=True)

#### Early Stopping

In [ ]:
early_stopping_model_2 = keras.callbacks.EarlyStopping(monitor="loss", min_delta=0.002, patience=2)

#### Konfiguration von Checkpoints

In [ ]:
checkpoint_dir = './work/training_checkpoints/gru_model_2'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}.weights.h5")

checkpoint_callback_gru_model_2=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

#### Ausführen des Trainings

In [ ]:
EPOCHS = 30

In [ ]:
start = time.perf_counter()
gru_model_2_history = gru_model_2.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback_gru_model_2, early_stopping_model_2])
end = time.perf_counter()
gru_model_2_training_time = end - start

### Speichern

In [ ]:
gru_model_2.save('work/models/gru_model_2.keras')

## Bewertung und Vergleich der GRU-Modelle

In [ ]:
loss1 = gru_model_1_history.history['loss']
loss2 = gru_model_2_history.history['loss']

accuracy1 = gru_model_1_history.history['accuracy']
accuracy2 = gru_model_2_history.history['accuracy']

perplexity1 = np.exp(loss1)
perplexity2 = np.exp(loss2)

fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(12, 4)) 

axes[0].plot(loss1, label="GRU Model 1", linestyle='-', color='blue')
axes[0].plot(loss2, label="GRU Model 2", linestyle='-', color='red')
axes[0].set_xlabel("Epochs")
axes[0].set_ylabel("Loss")
axes[0].set_title("Comparison of Training Histories")
axes[0].legend()

axes[1].plot(accuracy1, label="GRU Model 1", linestyle='-', color='blue')
axes[1].plot(accuracy2, label="GRU Model 2", linestyle='-', color='red')
axes[1].set_xlabel("Epochs")
axes[1].set_ylabel("Accuracy")
axes[1].set_title("Accuracy Comparison")
axes[1].legend()

axes[2].plot(perplexity1, label="GRU Model 1", linestyle='-', color='blue')
axes[2].plot(perplexity2, label="GRU Model 2", linestyle='-', color='red')
axes[2].set_xlabel("Epochs")
axes[2].set_ylabel("Perplexity")
axes[2].set_title("Perplexity Comparison")
axes[2].legend()

plt.tight_layout()

plt.show()

print("Loss:")
print(f" GRU Model 1: {loss1[-1]}")
print(f" GRU Model 2: {loss2[-1]}")
print("Accuracy:")
print(f" GRU Model 1: {accuracy1[-1]}")
print(f" GRU Model 2: {accuracy2[-1]}")
print("Perplexity:")
print(f" GRU Model 1: {perplexity1[-1]}")
print(f" GRU Model 2: {perplexity2[-1]}")
print("Training Times:")
print(f" GRU Model 1: {gru_model_1_training_time:2f}s")
print(f" GRU Model 2: {gru_model_2_training_time:2f}s")


### LSTM-Modell

In [ ]:
vocab_size_lstm = len(ids_from_chars.get_vocabulary())
embedding_dim_lstm = 256
rnn_units_lstm = 2048

In [ ]:

inputs_lstm = keras.layers.Input(shape=(None,), dtype='int32', name='input_tokens')
embedding_lstm = keras.layers.Embedding(input_dim=vocab_size_lstm, output_dim=embedding_dim_lstm)(inputs_lstm)
lstm, hidden_state_1, cell_state_1 = keras.layers.LSTM(units=rnn_units_lstm, return_sequences=True, return_state=True)(embedding_lstm)
lstm, hidden_state_2, cell_state_2 = keras.layers.LSTM(units=rnn_units_lstm, return_sequences=True, return_state=True)(lstm)
outputs_lstm = keras.layers.Dense(units=vocab_size, activation='softmax')(lstm)

lstm_model = keras.Model(inputs=inputs_lstm, outputs=outputs_lstm)

#### Testen des Modells

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions_lstm_model = lstm_model(input_example_batch)
    print(example_batch_predictions_lstm_model.shape, "# (batch_size, sequence_length, vocab_size)")

In [ ]:
lstm_model.summary()

In [ ]:
sampled_indices_lstm_model = tf.random.categorical(example_batch_predictions_lstm_model[0], num_samples=1)
sampled_indices_lstm_model = tf.squeeze(sampled_indices_lstm_model, axis=-1).numpy()
sampled_indices_lstm_model

In [ ]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices_gru_model_2).numpy())

### Training

#### Loss

In [ ]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
example_batch_mean_loss_lstm_model = loss(target_example_batch, example_batch_predictions_lstm_model)
print("Prediction shape: ", example_batch_predictions_lstm_model.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss_lstm_model)

In [ ]:
tf.exp(example_batch_mean_loss_lstm_model).numpy()

#### Optimizer

In [ ]:
lstm_model.compile(optimizer='adam', loss=loss, metrics=['accuracy'], run_eagerly=True)

#### Konfiguration von Checkpoints

In [ ]:
checkpoint_dir = './work/training_checkpoints/lstm_model'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}.weights.h5")

checkpoint_callback_gru_model_2=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

#### Ausführen des Trainings

In [ ]:
EPOCHS = 30

In [ ]:
start = time.perf_counter()
lstm_model_history = lstm_model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback_gru_model_2, early_stopping_model_2])
end = time.perf_counter()
lstm_model_training_time = end - start

#### Speichern

In [ ]:
lstm_model.save('work/models/lstm_model.keras')

## Vergleich aller Modelle

In [ ]:
loss1 = gru_model_1_history.history['loss']
loss2 = gru_model_2_history.history['loss']
loss3 = lstm_model_history.history['loss']

accuracy1 = gru_model_1_history.history['accuracy']
accuracy2 = gru_model_2_history.history['accuracy']
accuracy3 = lstm_model_history.history['accuracy']

perplexity1 = np.exp(loss1)
perplexity2 = np.exp(loss2)
perplexity3 = np.exp(loss3)

fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(12, 4)) 

axes[0].plot(loss1, label="GRU Model 1", linestyle='-', color='blue')
axes[0].plot(loss2, label="GRU Model 2", linestyle='-', color='red')
axes[0].plot(loss3, label="LSTM Model", linestyle='-', color='green')
axes[0].set_xlabel("Epochs")
axes[0].set_ylabel("Loss")
axes[0].set_title("Comparison of Training Histories")
axes[0].legend()

axes[1].plot(accuracy1, label="GRU Model 1", linestyle='-', color='blue')
axes[1].plot(accuracy2, label="GRU Model 2", linestyle='-', color='red')
axes[1].plot(accuracy3, label="LSTM Model", linestyle='-', color='green')
axes[1].set_xlabel("Epochs")
axes[1].set_ylabel("Accuracy")
axes[1].set_title("Accuracy Comparison")
axes[1].legend()

axes[2].plot(perplexity1, label="GRU Model 1", linestyle='-', color='blue')
axes[2].plot(perplexity2, label="GRU Model 2", linestyle='-', color='red')
axes[2].plot(perplexity3, label="LSTM Model", linestyle='-', color='green')
axes[2].set_xlabel("Epochs")
axes[2].set_ylabel("Perplexity")
axes[2].set_title("Perplexity Comparison")
axes[2].legend()

plt.tight_layout()

plt.show()
print("Loss:")
print(f" GRU Model 1: {loss1[-1]}")
print(f" GRU Model 2: {loss2[-1]}")
print(f" LSTM Model:  {loss3[-1]}")
print("Accuracy:")
print(f" GRU Model 1: {accuracy1[-1]}")
print(f" GRU Model 2: {accuracy2[-1]}")
print(f" LSTM Model:  {accuracy3[-1]}")
print("Perplexity:")
print(f" GRU Model 1: {perplexity1[-1]}")
print(f" GRU Model 2: {perplexity2[-1]}")
print(f" LSTM Model:  {perplexity3[-1]}")
print("Training Times:")
print(f" GRU Model 1: {gru_model_1_training_time:2f}s")
print(f" GRU Model 2: {gru_model_2_training_time:2f}s")
print(f" LSTM Model:  {lstm_model_training_time:2f}s")